# Scratch space for trying things out

In [1]:
%load_ext autoreload
%autoreload 2
#%matplotlib widget
#%matplotlib ipympl

#%reload_ext tensorboard
#%matplotlib qt

In [2]:
import os
from pprint import pprint
from pathlib import Path
from datetime import datetime
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from collections import OrderedDict
import SimpleITK as sitk
from collections import OrderedDict
import torch
import torchvision
from tqdm.auto import tqdm
import pickle, subprocess
import scipy
import sklearn
import csv

import torchmetrics
import glob

from hnc_project import data_prep as dp
from hnc_project import myshow

#%matplotlib notebook
%matplotlib widget
plt.ion()
#import initial_ml as iml

D:\work\Miniconda3\envs\pytorch_gpu_v2\Lib\site-packages\torchaudio\backend\utils.py:74: UserWarning: No audio backend is available.
  warnings.warn("No audio backend is available.")


In [3]:
data_directory = '../../data/HNSCC'
nii_directory = '../../data/HNSCC/HNSCC_Nii_v3'
resample_directory = '../../data/HNSCC/HNSCC_Nii_resample_222'
graph_directory = '../../data/HNSCC/graph_staging'
patch_directory = '../../data/HNSCC/HNSCC_Nii_222_50_50_60_Crop'
location_pickle = '../../data/HNSCC/HNSCC_Nii_222_50_50_60_Crop/locations.pkl'

patients_to_drop = [
    'HNSCC-01-0228',
    'HNSCC-01-0253',
    'HNSCC-01-0358',
    'HNSCC-01-0464',
]
data_path = Path(data_directory)
nii_path = Path(nii_directory)
resample_path = Path(resample_directory)
patch_path = Path(patch_directory)
pickle_path = Path(location_pickle)
resample_path.mkdir(exist_ok=True, parents=True)
patch_path.mkdir(exist_ok=True, parents=True)

In [ ]:
# get patients that at least have a GTVp mask in their directory
filled_dirs = [p for p in patient_dirs if np.any(['gtv' in f.lower() for f in glob.glob(f"{p}/*.nii*")])]

In [ ]:
# make dictionary of patients and available image files

gtvp_file_names = ['gtv.nii',
                   'gtvp.nii',
                   'gtv.nii',
                   'gtv2.nii',
                   'Primary-GTV.nii',
                   'GTV1.nii',
                   'gtv-primary',
                   'gtv-prechemo',
                   'gtv-postchemo',
                   'gtv-bot',
                   'GTV-Pre-CTH.nii',
                   'GTVTumorPre.nii',
                   'Pre-chemo-GTV',
                   'GTV-Tumor',
                   'GTV-T',
                   'pre-chemo-GTV',
                  ]
                   
gtvn_file_names = ['GTVn', 
                   'GTV-Nodes',
                   'GTV-NODES',
                   'GTV-node',
                   'Node-GTV',
                   'gtv-node',
                   'gtv-nodes',
                   'Nodal-GTV',
                   'nodal-gtv',
                   'GTV-LN',
                   'node-GTV',
                   'GTV-LT-node',
                   'GTV-Lt-nodes',
                   'GTV-Rt-Nodes',
                   'gtv-ln',
                   'L-LN-GTV',
                   'GTVNodesPre.nii',
                   'GTV-N',
                  ]
checks = ['GTV',
          'gtv',
          #'nod',
          #'Nod',
          #'NOD',
          #'LN',
         ]
                   
patient_dict = {p.split('\\')[-1]: [nii for nii in os.listdir(p) 
                                    if ('gtv' in nii.lower() or 'image' in nii) 
                                    and 'ptv' not in nii.lower() 
                                    and 'ctv' not in nii.lower()
                                    and 'expanded' not in nii.lower()
                                   ] 
                for p in filled_dirs
                if int(p.split('\\')[-1].split('-')[-1].split('_')[0]) > 215
                and p.split('\\')[-1].replace('.nii.gz','') not in patients_to_drop
               }

In [ ]:
for p in patient_dict:
    print('+++++++++++++++')
    print(p)
    for mask in patient_dict[p]:
        print(f'    {mask}')

In [ ]:
patient = 'HNSCC-01-0272_00'
masks = {}
struct_image = sitk.ReadImage(os.path.join(resample_directory, patient, patient_dict[patient][0]))
for i, m in enumerate(patient_dict[patient]):
    if 'image' in m: continue
    masks[m.split('.')[0]] = sitk.ReadImage(os.path.join(resample_directory, patient, m))
    
    

In [ ]:
sitk.Show(struct_image)
sitk.Show(masks['mask_GTVp'])

In [ ]:
patches = {}
for m in masks:
    patches[m] = np.where(sitk.GetArrayFromImage(masks[m])>0, sitk.GetArrayFromImage(struct_image), 0)

In [ ]:
for p in patient_dict:
    save_dir = os.path.join(patch_directory, p)
    if not os.path.exists(save_dir): 
        os.makedirs(save_dir) 
    struct = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(resample_directory, p, patient_dict[p][0])))
    for i, m in enumerate(patient_dict[p]):
        if 'image' in m: continue
        mask = sitk.GetArrayFromImage(sitk.ReadImage(os.path.join(resample_directory, p, m)))
        np.save(os.path.join(save_dir, m.replace('mask_','').split('.')[0]), np.where(mask>0, struct, 0))

In [ ]:
#patient = 'HNSCC-01-0272_00'
masks = {}
struct_image = sitk.ReadImage(os.path.join(nii_directory, patient, patient_dict[patient][0]))
for i, m in enumerate(patient_dict[patient]):
    if 'image' in m: continue
    masks[m.split('.')[0]] = sitk.ReadImage(os.path.join(nii_directory, patient, m))
    
    

In [ ]:
for p in resample_path.glob('*'):
    print(p.as_posix().split('/')[-1])

In [ ]:
shapes = {}
for p in resample_path.glob('*'):
    p_str = p.as_posix().split('/')[-1]
    if '0286' in p_str: continue
    for m in p.glob('*.nii.gz'):
        if 'image' in str(m): continue
        m_str = m.as_posix().split('/')[-1].replace('.nii.gz','').replace('mask_', '')
        mask = sitk.GetArrayFromImage(sitk.ReadImage(m))
        shapes[f'{p_str}_{m_str}'] = np.nonzero(mask)

In [ ]:
shape_size = []
for s in shapes:
    print(s)
    
    try:
        x = np.max(shapes[s][0]) - np.min(shapes[s][0])
        y = np.max(shapes[s][1]) - np.min(shapes[s][1])
        z = np.max(shapes[s][2]) - np.min(shapes[s][2])
        #shape_size[s] = [x, y, z]
        shape_size.append([x, y, z])
    except:
        patients_to_drop.append(s)
shape_size = np.array(shape_size)

In [ ]:
print(np.max(shape_size[:,0]))
print(np.max(shape_size[:,1]))
print(np.max(shape_size[:,2]))

## Resampling code

In [ ]:
resampler = sitk.ResampleImageFilter()
resampler.SetOutputDirection([1, 0, 0, 0, 1, 0, 0, 0, 1])
resampling = [2,2,2]
resampler.SetOutputSpacing(resampling)

In [ ]:
def get_bouding_boxes(ct, pt):
    """
    Get the bounding boxes of the CT and PT images.
    This works since all images have the same direction
    """

    ct_origin = np.array(ct.GetOrigin())
    pt_origin = np.array(pt.GetOrigin())

    ct_position_max = ct_origin + np.array(ct.GetSize()) * np.array(
        ct.GetSpacing())
    pt_position_max = pt_origin + np.array(pt.GetSize()) * np.array(
        pt.GetSpacing())
    return np.concatenate(
        [
            np.maximum(ct_origin, pt_origin),
            np.minimum(ct_position_max, pt_position_max),
        ],
        axis=0,
    )

In [ ]:
def resample_one_patient(p):
    pat_str = p.as_posix().split('/')[-1]
    patient_resample_path = resample_path.joinpath(pat_str)
    patient_resample_path.mkdir(exist_ok=True, parents=True)
    
    ct = sitk.ReadImage(p.joinpath('image.nii.gz'))
    #label = sitk.ReadImage(os.path.join(savePath, p, 'mask_GTVp.nii.gz'))
    bb = get_bouding_boxes(ct, ct)
    size = np.round((bb[3:] - bb[:3]) / resampling).astype(int)
    resampler.SetOutputOrigin(bb[:3])
    resampler.SetSize([int(k) for k in size])  # sitk is so stupid
    resampler.SetInterpolator(sitk.sitkBSpline)
    ct = resampler.Execute(ct)
    sitk.WriteImage(ct, patient_resample_path.joinpath('image.nii.gz'))
    resampler.SetInterpolator(sitk.sitkNearestNeighbor)
    for m in p.glob('*.nii.gz'):
        if 'image' in str(m): continue
        label = sitk.ReadImage(m)
        label = resampler.Execute(label)
        sitk.WriteImage(label, patient_resample_path.joinpath(str(m).split('\\')[-1]))

In [ ]:
for pat in tqdm(nii_path.glob('*')):
    pat_str = pat.as_posix().split('/')[-1]
    if pat_str in patients_to_drop: continue   
    resample_one_patient(pat)
    

## 4. Cropping

In [4]:
def find_centroid(mask, p):

    stats = sitk.LabelShapeStatisticsImageFilter()
    stats.Execute(mask)
    try:
        centroid_coords = stats.GetCentroid(1)
    except:
        print(f'Something is wrong with centroid calculation for patient: {p}')
    centroid_idx = mask.TransformPhysicalPointToIndex(centroid_coords)

    return np.asarray(centroid_idx, dtype=np.float64), np.asarray(centroid_coords, dtype=np.float64)

In [5]:
crop_path = Path('../../data/HNSCC/HNSCC_Nii_222_50_50_60_Crop_v2')
crop_path.mkdir(exist_ok=True, parents=True)

In [13]:
def tune_range(min_d, max_d, d, size_d, p):
    min_pad = 0
    max_pad = 0
    if min_d<0:
        min_pad = abs(min_d)
        min_d = 0
        #max_d = min_d + d
        #if max_d - size_d > 0:
        #    max_pad = max_d-size_d
            
        #assert (max_d<size_d), f"Cannot extract the patch with the shape {size_d} from the image with the shape {d} for patient {p}."
    
    if max_d>d:
        max_pad = max_d - d
        max_d = d
        #min_d = max_d - size_d
        #if min_d < 0:
        #    min_pad = abs(min_d)
            
        #assert (min_d>0), f"Cannot extract the patch with the shape {size_d} from the image with the shape {d} for patient {p}."

    return min_d, max_d, int(min_pad), int(max_pad)
patients_to_retry = [
    'HNSCC-01-0225',
    'HNSCC-01-0259',
    'HNSCC-01-0272',
    'HNSCC-01-0434',
]
physical_locations = {}
for p_dir in tqdm(resample_path.glob('*')):
    p_str = p_dir.as_posix().split('/')[-1]
    #if p_str not in patients_to_retry: continue
    #try:
    if p_str in patients_to_drop:
        print('skip ', p_str)
        continue
    patient_crop_path = crop_path.joinpath(p_str)
    patient_crop_path.mkdir(exist_ok=True, parents=True)
    physical_locations[p_str] = {}
    patch_size = np.array([50,50,60])
    for m in p_dir.glob('*.nii.gz'):
        print('-----------------')
        m_str = m.as_posix().split('/')[-1]
        if 'image' in m_str: continue
        #try:
        image = sitk.ReadImage(p_dir.joinpath('image.nii.gz'))
        mask = sitk.ReadImage(m)
        #crop the image to patch_size around the tumor center
        tumour_center, center_location = find_centroid(mask, p_str) # center of GTV
        size = patch_size
        min_coords = np.floor(tumour_center - size / 2).astype(np.int64)
        max_coords = np.floor(tumour_center + size / 2).astype(np.int64)
        min_x, min_y, min_z = min_coords
        max_x, max_y, max_z = max_coords
        (img_x, img_y, img_z)=image.GetSize()
        min_x, max_x, min_pad_x, max_pad_x = tune_range(min_x, max_x, img_x, size[0], p_str) 
        min_y, max_y, min_pad_y, max_pad_y = tune_range(min_y, max_y, img_y, size[1], p_str) 
        min_z, max_z, min_pad_z, max_pad_z = tune_range(min_z, max_z, img_z, size[2], p_str) 

        min_pad = int(max([min_pad_x, min_pad_y, min_pad_z]))
        max_pad = int(max([max_pad_x, max_pad_y, max_pad_z]))
        lpad = list([min_pad_x, min_pad_y, min_pad_z])
        upad = list([max_pad_x, max_pad_y, max_pad_z])
        #print(m_str)
        #print(lpad)
        #print(upad)
        print(image.GetSize())
        print(min_coords, max_coords)
        print(min_pad, max_pad)
        image = image[min_x:max_x, min_y:max_y, min_z:max_z]
        # window image intensities to [-500, 1000] HU range
        image = sitk.Clamp(image, sitk.sitkFloat32, -500, 500)
        mask = mask[min_x:max_x, min_y:max_y, min_z:max_z]
        print(image.GetSize())
        image = sitk.ConstantPad(image, lpad, upad, 0.0)
        mask = sitk.ConstantPad(mask, lpad, upad, 0.0)
        print(image.GetSize())
        sitk.WriteImage(image, patient_crop_path.joinpath(f"image_{m_str.replace('Struct_','')}"))
        sitk.WriteImage(mask, patient_crop_path.joinpath(m_str))
        physical_locations[p_str][m_str.replace('Struct_','').replace('.nii.gz','')] = center_location
        del(image)
        del(mask)
        #except:
        #    print(m)
        #    raise Exception('something went wrong...')
    
    #except:
    #    print(p_str)
        
with open(crop_path.joinpath('locations.pkl'), 'wb') as f:
    pickle.dump(physical_locations, f)
    f.close()

0it [00:00, ?it/s]

-----------------
-----------------
(125, 125, 144)
[47 30 74] [ 97  80 134]
0 0
(50, 50, 60)
(50, 50, 60)
-----------------
-----------------
(125, 125, 147)
[20 25 42] [ 70  75 102]
0 0
(50, 50, 60)
(50, 50, 60)
-----------------
(125, 125, 147)
[35 27 48] [ 85  77 108]
0 0
(50, 50, 60)
(50, 50, 60)
-----------------
-----------------
(125, 125, 120)
[55 36 49] [105  86 109]
0 0
(50, 50, 60)
(50, 50, 60)
-----------------
(125, 125, 120)
[50 35 36] [100  85  96]
0 0
(50, 50, 60)
(50, 50, 60)
-----------------
(125, 125, 120)
[51 35 28] [101  85  88]
0 0
(50, 50, 60)
(50, 50, 60)
-----------------
(125, 125, 120)
[44 27 53] [ 94  77 113]
0 0
(50, 50, 60)
(50, 50, 60)
-----------------
-----------------
(125, 125, 172)
[12 38 39] [62 88 99]
0 0
(50, 50, 60)
(50, 50, 60)
-----------------
(125, 125, 172)
[14 35 32] [64 85 92]
0 0
(50, 50, 60)
(50, 50, 60)
-----------------
(125, 125, 172)
[29 28 35] [79 78 95]
0 0
(50, 50, 60)
(50, 50, 60)
-----------------
-----------------
(125, 125, 

In [25]:
test_image = sitk.ReadImage(patch_path.joinpath('HNSCC-01-0216/image_GTVp.nii.gz'))
test_struct = sitk.ReadImage(patch_path.joinpath('HNSCC-01-0216/Struct_GTVp.nii.gz'))
test_norm = sitk.NormalizeImageFilter()
test_image_norm = test_norm.Execute(test_image)
test_img_array = sitk.GetArrayFromImage(test_image_norm)
test_struct_array = sitk.GetArrayFromImage(test_struct)

In [34]:
np.shape(np.stack((test_img_array, test_struct_array)))

(2, 60, 50, 50)

In [ ]:
patient_patch_paths = patch_path.glob('*/')
tumor_locations = pd.read_pickle(pickle_path)
centered_locations = {}
for pat in patient_patch_paths:
    pat_str = pat.as_posix().split('/')[-1]
    centered_locations[pat_str] = {}
    n_tumors = len(tumor_locations[pat_str])
    translation_factor = np.array([0, 0, 0])
    if n_tumors == 1:
        centered_locations[pat_str]['GTVp'] = np.array([0., 0., 0.])
        continue
    else:
        if 'GTVp' in tumor_locations[pat_str].keys():
            translation_factor = tumor_locations[pat_str]['GTVp']
        else:
            array_locs = np.array([val for val in tumor_locations[pat_str].values()])
            origin_idx = np.where(array_locs == np.max(array_locs, axis=0)[2])[0][0]
            translation_factor = array_locs[origin_idx]
    for tumor in tumor_locations[pat_str]:
        centered_locations[pat_str][tumor.replace('.nii.gz','')] = tumor_locations[pat_str][tumor] - translation_factor

print(centered_locations)
                
        
    

In [ ]:
# r = sqrt(x^2 + y^2 + z^2)
# theta = atan2 ( sqrt(x^2+y^2) / z) accounting for different quadrants (make sure to use atan2 not atan)
# phi = atan2 (y/x) 

spherical_locations = {}
for pat, locs in centered_locations.items():
    spherical_locations[pat] = {}
    for gtv, l in locs.items():
        if np.all([not(l[0]), not(l[1]), not(l[2])]): 
            print(f'origin: {pat}, {gtv}')
            spherical_locations[pat][gtv] = np.array([0.,0.,0.])
            continue
        radius = np.sqrt(l[0]**2+l[1]**2+l[2]**2)
        theta = np.arctan2(np.sqrt(l[0]**2+l[1]**2), l[2])
        phi = np.arctan2(l[1], l[0])
        if phi < 0:
            phi = phi + 2*np.pi
        spherical_locations[pat][gtv] = np.array([radius, theta, phi])

print(spherical_locations)
pat_259 = spherical_locations['HNSCC-01-0259']
#with open(crop_path.joinpath('sph_locations.pkl'), 'wb') as f:
#    pickle.dump(spherical_locations, f)
#    f.close()

In [ ]:
import csv
with open(crop_path.joinpath('sph_locations.csv'), 'w', newline='') as f:
    writer = csv.writer(f)
    for pat, gtv in spherical_locations.items():
        for gtv_str, coords in gtv.items():
            writer.writerow([pat, gtv_str, coords])
            


In [ ]:
groups_phi_rtheta

In [ ]:
pat_259_r_sorted = {k: v for k, v in sorted(pat_259.items(), key=lambda item: item[1][0])}
pat_259_phi_sorted = {k: v for k, v in sorted(pat_259.items(), key=lambda item: item[1][2])}
pat_259_theta_sorted = {k: v for k, v in sorted(pat_259.items(), key=lambda item: item[1][1])}
groups_phi = {}
groups_phi_gtv = {}
reference = list(pat_259_phi_sorted.values())[1][2] 
g_num = 'phi_1'
groups_phi[g_num] = []
for gtv, l in pat_259_phi_sorted.items():
    if 0. in l: 
        groups_phi_gtv[gtv] = 'phi_0'
        continue
    phi_diff = l[2] - reference
    print(phi_diff)
    if np.abs(phi_diff) < 0.4:
        groups_phi[g_num].append(gtv)
        groups_phi_gtv[gtv] = g_num
    else:
        g_num = f"phi_{int(g_num.split('_')[-1])+1}"
        groups_phi[g_num] = []
        groups_phi[g_num].append(gtv)
        groups_phi_gtv[gtv] = g_num
        reference = l[2]
for g in groups_phi:
    groups_phi[g] = [k for k in sorted(groups_phi[g], key=lambda gtv: pat_259[gtv][0])] 
    
groups_theta = {}
groups_theta_gtv = {}
reference = list(pat_259_theta_sorted.values())[1][1] 
g_num = 'theta_1'
groups_theta[g_num] = []
print(pat_259_theta_sorted)
for gtv, l in pat_259_theta_sorted.items():
    if 0. in l:
        groups_theta_gtv[gtv] = 'theta_0'
        continue
    theta_diff = l[1] - reference
    print(gtv)
    print(theta_diff)
    if np.abs(theta_diff) < 0.3:
        groups_theta[g_num].append(gtv)
        groups_theta_gtv[gtv] = g_num
    else:
        g_num = f"theta_{int(g_num.split('_')[-1])+1}"
        groups_theta[g_num] = []
        groups_theta[g_num].append(gtv)
        groups_theta_gtv[gtv] = g_num
        reference = l[1]
for g in groups_theta:
    groups_theta[g] = [k for k in sorted(groups_theta[g], key=lambda gtv: pat_259[gtv][0])]
print(groups_phi)
print(groups_theta)
print(groups_phi_gtv)
print(groups_theta_gtv)

In [ ]:
df_groups_phi = pd.DataFrame(groups_phi_gtv.values(), index=groups_phi_gtv.keys(), columns=['phi_group'])
df_groups_theta = pd.DataFrame(groups_theta_gtv.values(), index=groups_theta_gtv.keys(), columns=['theta_group'])
print(df_groups_phi)
print(df_groups_theta)

In [ ]:
pat_259 = spherical_locations['HNSCC-01-0259']
#plt.ion()
fig = plt.figure()
ax = fig.add_subplot(projection='3d')
points = []
label = []
for gtv in pat_259:
    r = pat_259[gtv][0]
    theta = pat_259[gtv][1]
    phi = pat_259[gtv][2]
    x = r * np.sin(theta) * np.cos(phi)
    y = r * np.sin(theta) * np.sin(phi)
    z = r * np.cos(theta)
    points.append([x, y, z])
    label.append(gtv)

points = np.array(points)
ax.scatter(points[:,0], points[:,1], points[:,2])
for i, l in enumerate(label):
    ax.text(points[i, 0], points[i, 1], points[i, 2], l)
plt.show()
   

In [ ]:
from sklearn.cluster import AgglomerativeClustering
from scipy.cluster.hierarchy import dendrogram

hier_clust = AgglomerativeClustering(distance_threshold=0, n_clusters=None)
hier_clust = hier_clust.fit(df_259[['x', 'y', 'z']])

In [ ]:
def plot_dendrogram(model, **kwargs):
    # Create linkage matrix and then plot the dendrogram

    # create the counts of samples under each node
    counts = np.zeros(model.children_.shape[0])
    n_samples = len(model.labels_)
    for i, merge in enumerate(model.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [model.children_, model.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    dendrogram(linkage_matrix, **kwargs)
    return linkage_matrix

In [ ]:
linker = plot_dendrogram(hier_clust, show_leaf_counts=False, labels=df_259.index)
plt.show()

In [ ]:
print(hier_clust.children_)
print(hier_clust.distances_)

In [ ]:
N = len(df_259)
def create_node_tree(children):
    def recursive_walk(child):
        node = []
        for ch in child:
            if ch < N:
                node.append(df_259.index[ch])
            else:
                node.append(recursive_walk(children[ch-N]))

        return node

    all_nodes = []
    for idx in children:
        all_nodes.append(recursive_walk(idx))
    return all_nodes

test = create_node_tree(hier_clust.children_)
print(test)
    

In [ ]:
test

In [ ]:
connections = test[-1]
connections[1]
#df_259.loc[np.ndarray.flatten(np.array(connections[1][1]))]

In [ ]:
connections

In [ ]:
def create_edges(tree):
    dict_edges = {}
    
    def recursive_walk(sub_tree, branch):
        for i, sub in enumerate(sub_tree):
            sub_branch = branch + str(i)
            if len(sub) == 2:
                dict_edges[sub_branch] = sub
                recursive_walk(sub, sub_branch)
            else:
                dict_edges[sub_branch] = sub

    for i, leaf in enumerate(tree):
        dict_edges[str(i)] = leaf
        recursive_walk(leaf, str(i))
    return dict_edges

In [ ]:
terminals = []
test_edges = create_edges(connections)
for branch in test_edges:
    if len(test_edges[branch]) == 2: continue
    else:
        terminals.append(branch)
terminals_map = {v: k for k, v in test_edges.items() if k in terminals}
edges_map = {str(v): k for k, v in test_edges.items()}
print(terminals)
print(test_edges)
print(terminals_map)

In [ ]:
test_edges_remap = {}
for k, v in test_edges.items():
    test_edges_remap[k] = []
    if k in terminals:
        test_edges_remap[k].append(test_edges[k])
    else:
        for arr in v:
            test_edges_remap[k].append(edges_map[str(arr)])
#test_edges_remap = {k: v for k, v in sorted(test_edges_remap.items(), key=lambda item: len(item[0]), reverse=True)}
test_edges_remap = {k: v for k, v in sorted(test_edges_remap.items(), key=lambda item: len(item[0]))}
test_edges_remap = {'p': ['0', '1'], **test_edges_remap}
#test_edges_remap = {**test_edges_remap, 'p': [0,1]}
test_edges = {k: v for k, v in sorted(test_edges.items(), key=lambda item: len(item[0]))}
test_edges_remap

In [ ]:
edges = []
skip_to_primary = []
def get_tree(branch):
    def recursive_walk(sub_branch):
        global skip_to_primary
        compare = []
        sub_branch_leaves = test_edges_remap[sub_branch]
        for leaf in sub_branch_leaves:
            print(f"        sub leaf: {leaf} for branch: {sub_branch}")
            if leaf in test_edges_remap.keys():
                print(f"    entering recursion for {leaf} in {sub_branch}")
                compare.append(recursive_walk(leaf))
            else:
                compare.append(leaf)

        if np.all([isinstance(c, str) for c in compare]):
            print(compare)
            df_c = df_259.loc[compare]
            if len(df_c) == 1:
                print(f"    exiting recursion for {sub_branch}")
                return df_c.index[0]
            df_c.sort_values(['r'], inplace=True)
            print(f"adding edge {[df_c.index[0], df_c.index[1]]}")
            edges.append([df_c.index[0], df_c.index[1]])
            print(f"    exiting recursion for {sub_branch}")
            return list(df_c.index)
            
        to_return = []
        for i, leaf in enumerate(compare):
            if isinstance(leaf, str):
                compare[i] = [leaf]
            
        df_0 = df_259.loc[compare[0]]
        df_0.sort_values('r', inplace=True)
        df_1 = df_259.loc[compare[1]]
        df_1.sort_values('r', inplace=True)

        if len(df_0.index) == 2:
            if [df_0.index[0], df_0.index[1]] not in edges:
                edges.append([df_0.index[0], df_0.index[1]])

        if len(df_1.index) == 2:
            if [df_1.index[0], df_1.index[1]] not in edges:
                print(f"adding edge: {[df_1.index[0], df_1.index[1]]}")
                edges.append([df_1.index[0], df_1.index[1]])
                
        if df_1['theta_group'].nunique(0) == 1:
            if df_0.loc[df_0.index[-1]]['r'] < df_1.loc[df_1.index[0]]['r']:
                print(f"adding edge: {[df_0.index[-1], df_1.index[0]]}")
                edges.append([df_0.index[-1], df_1.index[0]])
            elif df_1.loc[df_1.index[-1]]['r'] < df_0.loc[df_0.index[0]]['r']:
                print(f"adding edge: {[df_1.index[-1], df_0.index[0]]}")
                edges.append([df_1.index[-1], df_0.index[0]])
        elif df_0['theta_group'].nunique(0) == 1:
            if df_0.loc[df_0.index[-1]]['r'] < df_1.loc[df_1.index[0]]['r']:
                print(f"adding edge: {[df_0.index[-1], df_1.index[0]]}")
                edges.append([df_0.index[-1], df_1.index[0]])
            elif df_1.loc[df_1.index[-1]]['r'] < df_0.loc[df_0.index[0]]['r']:
                print(f"adding edge: {[df_1.index[-1], df_0.index[0]]}")
                edges.append([df_1.index[-1], df_0.index[0]])
                
        if np.max(df_0['r']) < np.max(df_1['r']):
            for leaf in df_1.index:
                if df_1['phi_group'].nunique(0) == 1 and df_1['theta_group'].nunique(0) > 1:
                    if [df_0.index[-1], leaf] in edges: continue
                    print(f"adding edge: {[df_0.index[-1], leaf]}")
                    edges.append([df_0.index[-1], leaf]) 
        else:
            for leaf in df_0.index:
                if df_0['phi_group'].nunique(0) == 1 and df_0['theta_group'].nunique(0) > 1:
                    if [df_1.index[-1], leaf] in edges: continue
                    print(f"adding edge: {[df_1.index[-1], leaf]}")
                    edges.append([df_1.index[-1], leaf]) 

        
        df_sub_branch = df_259.loc[compare[0] + compare[1]]
        df_sub_branch.sort_values(['phi_group', 'r'], inplace=True)

        if df_sub_branch['phi_group'].nunique(0) >  1:
            phi_counts = df_sub_branch.value_counts('phi_group')
            group_to_send = phi_counts[phi_counts == phi_counts.min()].index[0]
            skip_to_primary += list(df_sub_branch[df_sub_branch['phi_group']==group_to_send].index)
            
            
        return list(df_sub_branch.index[:2])
        
            
                

    to_compare = []
    leaves = test_edges_remap[branch]
    for leaf in leaves:
        print(f"leaf: {leaf}")
        if leaf in test_edges_remap.keys():
            print(f"    entering recursion for {leaf}")
            to_compare.append(recursive_walk(leaf))
        else:
            to_compare.append([leaf])

    print(f"done with recursion, for branch: {branch}")
    last_compare = []
    for i, leaf in enumerate(to_compare):
        if isinstance(leaf, str):
            to_compare[i] = [leaf]
    df_0 = df_259.loc[to_compare[0]]
    df_0.sort_values('r', inplace=True)
    df_1 = df_259.loc[to_compare[1]]
    df_1.sort_values('r', inplace=True)

    if df_1['theta_group'].nunique(0) == 1:
        if df_0.loc[df_0.index[-1]]['r'] < df_1.loc[df_1.index[0]]['r']:
            print(f"adding edge: {[df_0.index[-1], df_1.index[0]]}")
            edges.append([df_0.index[-1], df_1.index[0]])
        elif df_1.loc[df_1.index[-1]]['r'] < df_0.loc[df_0.index[0]]['r']:
            print(f"adding edge: {[df_1.index[-1], df_0.index[0]]}")
            edges.append([df_1.index[-1], df_0.index[0]])
    elif df_0['theta_group'].nunique(0) == 1:
        if df_0.loc[df_0.index[-1]]['r'] < df_1.loc[df_1.index[0]]['r']:
            print(f"adding edge: {[df_0.index[-1], df_1.index[0]]}")
            edges.append([df_0.index[-1], df_1.index[0]])
        elif df_1.loc[df_1.index[-1]]['r'] < df_0.loc[df_0.index[0]]['r']:
            print(f"adding edge: {[df_1.index[-1], df_0.index[0]]}")
            edges.append([df_1.index[-1], df_0.index[0]])
            
    if np.max(df_0['r']) < np.max(df_1['r']):
        for leaf in df_1.index:
            if df_1['phi_group'].nunique(0) == 1 and df_1['theta_group'].nunique(0) > 1:
                if [df_0.index[-1], leaf] in edges: continue
                print(f"adding edge: {[df_0.index[-1], leaf]}")
                edges.append([df_0.index[-1], leaf]) 
    else:
        for leaf in df_0.index:
            if df_0['phi_group'].nunique(0) == 1 and df_0['theta_group'].nunique(0) > 1:
                if [df_1.index[-1], leaf] in edges: continue
                print(f"adding edge: {[df_1.index[-1], leaf]}")
                edges.append([df_1.index[-1], leaf]) 
    
    df_branch = df_259.loc[to_compare[0] + to_compare[1]]
    df_branch.sort_values('r', inplace=True)
    return list(df_branch.index[:2])


final_compare = []
for leaf in test_edges_remap['p']:
    final_compare.append(get_tree(leaf))
print(f"finished tree recursion, starting final comparisons")
last_edges = []
for c in final_compare:
    df_c = df_259.loc[c]
    df_c.sort_values('r', inplace=True)
    if df_c['phi_group'].nunique(0) == 1 and df_c['theta_group'].nunique(0) == 1:
        last_edges += [df_c.index[0]]
    else:
        last_edges += list(df_c.index)

for gtv in last_edges:
    print(f"adding edge: {[primary, gtv]}")
    edges.append([primary, gtv]) 
for gtv in skip_to_primary:
    print(f"adding edge: {[primary, gtv]}")
    edges.append([primary,gtv])
print(edges)

In [ ]:
test = df_259.value_counts('phi_group')

In [ ]:
test2 = []
test2 += list(test[test==test.min()].index)
print(test2)

In [ ]:
model.trainers = []
for idx in range(5):
    model.set_model()
    model.set_callbacks(idx)
    model.trainers.append(L.Trainer(
                    max_epochs=model.config['n_epochs'],
                    accelerator="auto",
                    devices=model.config['gpu_device'] if torch.cuda.is_available() else None,
                    logger=[L.loggers.CSVLogger(save_dir=os.path.join(model.log_dir, f"csvlog_fold_{idx}")), L.loggers.TensorBoardLogger(save_dir=os.path.join(model.log_dir, f"tb_fold_{idx}"))],
                    callbacks=model.callbacks,
                    #check_val_every_n_epoch = 1,
                    #auto_lr_find=True
                    ))

In [ ]:
## w/o graph models
model_directory = './logs/lightning_feat64_no_graph_weight9_real_dp3_11180_minmax_rot10_balance_newclinical_nolrfinder_nocensor_v36'
fold_models = [
    'model_m_epoch=46_val_loss=3.70_val_auc=0.89_val_m=0.86.ckpt',
    'model_m_epoch=01_val_loss=4.82_val_auc=0.72_val_m=0.65.ckpt',
    'model_m_epoch=52_val_loss=5.66_val_auc=0.88_val_m=0.81.ckpt',
    'model_m_epoch=10_val_loss=1.05_val_auc=0.81_val_m=0.72.ckpt',
    'model_m_epoch=30_val_loss=50.76_val_auc=0.68_val_m=0.71.ckpt',
]
## w/graph models
#model_directory = './logs/lightning_feat64_with_graph_weight9_real_dp3_11180_minmax_rot10_balance_newclinical_nolrfinder_nocensor_v36'
#fold_models = [
#    'model_m_epoch=28_val_loss=1.80_val_auc=0.87_val_m=0.68.ckpt',
#    'model_m_epoch=09_val_loss=0.42_val_auc=0.93_val_m=0.89.ckpt',
#    'model_m_epoch=15_val_loss=0.80_val_auc=0.87_val_m=0.85.ckpt',
#    'model_m_epoch=65_val_loss=0.36_val_auc=0.82_val_m=0.75.ckpt',
#    'model_m_epoch=00_val_loss=2.60_val_auc=0.68_val_m=0.67.ckpt',
#]

In [ ]:
for idx in range(5):
    model_path = os.path.join(model_directory, f"top_models_fold_{idx}", fold_models[idx])
    model.trainers[idx].validate(model.model, datamodule=model.data_module_cross_val[idx], ckpt_path=model_path)
    model.trainers[idx].test(model.model, datamodule=model.data_module_cross_val[idx], ckpt_path=model_path)
    